
# Actividad semana 7: Uso de redes neuronales recurrentes

Corporación universitaria Minuto de Dios<br>
Especialización en inteligencia artificial

**Juan Sebastián Ordoñez Acuña**<br>
**Bryan Sthefen Gomez Salamanca**

Procesamiento de lenguaje natural<br>
Claudia Marcela Ospina Mosquera


19 de Agosto de 2023


# Modelos del lenguaje con RNNs

En esta parte, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizará el modelo para generar texto. En particular,su modelo tiene obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), tener presente que GPU no funciona tan bien con LSTM, por tanto el modelo se debe generar con tiempo. 

El dataset a utilizar contiene un archivo de texto con el contenido del castellano antiguo de **El Ingenioso Hidalgo Don Quijote de la Mancha**, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). 

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)



Primero, se descarga el libro y se inspeccionan los datos. El fichero a descargar es una versión en .txt del libro de Don Quijote, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

<div style="color:white;
       display:fill;
       border-radius:5px;
       font-size:200%;
       font-family:Avenir;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:black;">
        1. Importacion de librerias
</div>



In [1]:
# propósito general
import numpy as np 
import keras
import matplotlib.pyplot as plt

# creación y entrenamiento del modelo
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM

# ajustes del sistema 
import sys
import random
import io

<div style="color:white;
       display:fill;
       border-radius:5px;
       font-size:200%;
       font-family:Avenir;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:black;">
        2. Carga de corpus de trabajo
</div>

Se realiza la carga del archivo y se convierte a minuscula

In [2]:
# Con la función open de Python se cargará el texto codificado en formato utf8
with open('don_quijote.txt', encoding="utf8") as file:
     # Se define variable para cargar el texto "text" con la función .read y se específica que lo haga en minúsculas
    text = file.read().lower()
# Mostrar primeros 1000 carácteres del texto para su comprobación
print('longuitud del texto', len(text))
print(text[0:1000])

longuitud del texto 2071198
capítulo primero. que trata de la condición y ejercicio del famoso hidalgo
don quijote de la mancha


en un lugar de la mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor. una olla de algo más vaca que carnero,
salpicón las más noches, duelos y quebrantos los sábados, lantejas los
viernes, algún palomino de añadidura los domingos, consumían las tres
partes de su hacienda. el resto della concluían sayo de velarte, calzas de
velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de
entresemana se honraba con su vellorí de lo más fino. tenía en su casa una
ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte,
y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la
podadera. frisaba la edad de nuestro hidalgo con los cincuenta años; era de
complexión recia, seco de carnes, enjuto de rostro, gran madrugador y 

<div style="color:white;
       display:fill;
       border-radius:5px;
       font-size:200%;
       font-family:Avenir;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:black;">
        3. Procesamiento del corpus
</div>

Una de las grandes ventajas de trabajar con modelos que utilizan caracteres en vez de palabras es que no necesita tokenizar el texto (partirlo palabra a palabra). Su modelo funcionará directamente con los caracteres en el texto, incluyendo espacios, saltos de línea, etc.

Como sabe, un modelo del lenguaje con RNNs acepta una serie de caracteres y predice el siguiente carácter en la secuencia.

* "*El ingenioso don Qui*" -> predicción: **j**
* "*El ingenioso don Quij*" -> predicción: **o**

De modo que la entrada y la salida de su modelo necesita ser algo parecido a este esquema. En este punto, podría usar dos formas de preparar los datos para este modelo.

1. **Secuencia a secuencia**. La entrada del modelo sería una secuencia y la salida sería esa secuencia trasladada un caracter a la derecha, de modo que en cada instante de tiempo la RNN tiene que predecir el carácter siguiente. Por ejemplo:

>* *Input*:   El ingenioso don Quijot 
>* *Output*: l ingenioso don Quijote

2. **Secuencia a carácter**. En este variante, pasa una secuencia de caracteres por la RNN y, al llegar al final de la secuencia, se predice el siguiente carácter.

>* *Input*:   El ingenioso don Quijot 
>* *Output*: e

En este trabajo, por simplicidad, se utilizará la segunda variante.
Utilice secuencias de tamaño *SEQ_LENGTH* caracteres (y elija el hiperparámetro ).



<div style="color:white;
       display:fill;
       border-radius:5px;
       font-size:150%;
       font-family:Avenir;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:black;">
        3.1. Obtención de los caracteres y mapas de caracteres
</div>

Antes que nada, se necesita saber qué caracteres aparecen en el texto, ya que tiene que diferenciarlos mediante un índice de 0 a *num_chars* - 1 en el modelo. Obtener:
 

1.   Número de caracteres únicos que aparecen en el texto.
2.   Diccionario que asocia char a índice único entre 0 y *num_chars* - 1. Por ejemplo, {'a': 0, 'b': 1, ...}
3.   Diccionario reverso de índices a caracteres: {0: 'a', 1: 'b', ...}


In [3]:
# se crea variable para obtener una lista de los caracteres únicos presentes en el texto
chars=sorted(list(set(text)))

# se crea variable para asignar a cada caracter único del texto un índice (diccionario caracter - )
char_indices = dict((c,i) for i, c in enumerate(chars))

# se asigna a cada índice su caracter correspondiente (diccionario reverso)
indices_char = dict((i, c) for i, c in enumerate(chars))

# mostrar número total de caracteres únicos en el texto
len(chars) # se obtienen 61 caracteres

61

In [4]:
# ver lista de caracteres obtenida
chars

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 ':',
 ';',
 '?',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '¡',
 '«',
 '»',
 '¿',
 'à',
 'á',
 'é',
 'í',
 'ï',
 'ñ',
 'ó',
 'ù',
 'ú',
 'ü']

In [5]:
# ver diccionario de caracter - índice
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 ':': 18,
 ';': 19,
 '?': 20,
 ']': 21,
 'a': 22,
 'b': 23,
 'c': 24,
 'd': 25,
 'e': 26,
 'f': 27,
 'g': 28,
 'h': 29,
 'i': 30,
 'j': 31,
 'l': 32,
 'm': 33,
 'n': 34,
 'o': 35,
 'p': 36,
 'q': 37,
 'r': 38,
 's': 39,
 't': 40,
 'u': 41,
 'v': 42,
 'w': 43,
 'x': 44,
 'y': 45,
 'z': 46,
 '¡': 47,
 '«': 48,
 '»': 49,
 '¿': 50,
 'à': 51,
 'á': 52,
 'é': 53,
 'í': 54,
 'ï': 55,
 'ñ': 56,
 'ó': 57,
 'ù': 58,
 'ú': 59,
 'ü': 60}

<div style="color:white;
       display:fill;
       border-radius:5px;
       font-size:150%;
       font-family:Avenir;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:black;">
        3.2. Obtención de secuencias de entrada y caracteres a predecir
</div>

Ahora, obtenga la secuencias de entrada en formato texto y los correspondientes caracteres a predecir. Para ello, recorra el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH caracteres y el siguiente caracter a predecir. Una vez hecho, mueva un carácter a la izquierda y hacer lo mismo para obtener una nueva secuencia y predicción. Guarde las secuencias en una variable ***sequences*** y los caracteres a predecir en una variable ***next_chars***.

Por ejemplo, si el texto fuera "Don Quijote" y SEQ_LENGTH fuese 5, tendríamos

* *sequences* = ["Don Q", "on Qu", "n Qui", " Quij", "Quijo", "uijot"]
* *next_chars* = ['u', 'i', 'j', 'o', 't', 'e']

In [6]:
# Definia el tamaño de las secuencias. Puede dejar este valor por defecto.
SEQ_LENGTH = 40
step = 3
# En esta variable "rawX" se colectarán las oraciones (sentences) creadas con los parámetros previamente estípulados
rawX = [] # sentences
# En esta variable "rawY" se colectarán los caracteres que siguen al final de cada oración creada en "rawX"
rawy = [] # next_chars

# crear función en búcle para obtener las oraciones y "próximo caracter"
for i in range(0, len(text) - SEQ_LENGTH, step):
    rawX.append(text[i: i+SEQ_LENGTH])
    rawy.append(text[i+SEQ_LENGTH])

Indicar el tamaño del training set que acabamos de generar.

In [7]:
# mostrar lista de sentencias creadas
rawX

['capítulo primero. que trata de la condic',
 'ítulo primero. que trata de la condición',
 'lo primero. que trata de la condición y ',
 'primero. que trata de la condición y eje',
 'mero. que trata de la condición y ejerci',
 'o. que trata de la condición y ejercicio',
 'que trata de la condición y ejercicio de',
 ' trata de la condición y ejercicio del f',
 'ata de la condición y ejercicio del famo',
 ' de la condición y ejercicio del famoso ',
 ' la condición y ejercicio del famoso hid',
 ' condición y ejercicio del famoso hidalg',
 'ndición y ejercicio del famoso hidalgo\nd',
 'ción y ejercicio del famoso hidalgo\ndon ',
 'n y ejercicio del famoso hidalgo\ndon qui',
 ' ejercicio del famoso hidalgo\ndon quijot',
 'ercicio del famoso hidalgo\ndon quijote d',
 'icio del famoso hidalgo\ndon quijote de l',
 'o del famoso hidalgo\ndon quijote de la m',
 'el famoso hidalgo\ndon quijote de la manc',
 'famoso hidalgo\ndon quijote de la mancha\n',
 'oso hidalgo\ndon quijote de la mancha\n\n\n

In [8]:
# mostrar lista de "próximos caracteres"
rawy

['i',
 ' ',
 'e',
 'r',
 'c',
 ' ',
 'l',
 'a',
 's',
 'h',
 'a',
 'o',
 'o',
 'q',
 'j',
 'e',
 'e',
 'a',
 'a',
 'h',
 '\n',
 'n',
 'n',
 'u',
 'r',
 'e',
 'a',
 'a',
 'h',
 ' ',
 ' ',
 'y',
 'n',
 'b',
 ' ',
 ' ',
 'i',
 'o',
 'c',
 'd',
 'm',
 ' ',
 ' ',
 ' ',
 'c',
 '\n',
 'e',
 'o',
 'u',
 'v',
 'í',
 'u',
 'h',
 'a',
 'o',
 'e',
 'o',
 'd',
 'l',
 'z',
 'e',
 'a',
 'i',
 'e',
 ',',
 'd',
 'g',
 'a',
 'i',
 'a',
 'r',
 'í',
 'f',
 'c',
 'y',
 'a',
 'o',
 'o',
 'e',
 'r',
 'u',
 ' ',
 'l',
 'd',
 'a',
 'o',
 'á',
 'v',
 'a',
 'u',
 'c',
 'n',
 'o',
 's',
 'p',
 'ó',
 'l',
 ' ',
 's',
 'o',
 'e',
 ' ',
 'e',
 's',
 ' ',
 'e',
 'a',
 'o',
 'l',
 ' ',
 'b',
 'o',
 ' ',
 'n',
 'j',
 ' ',
 's',
 'i',
 'n',
 ',',
 'l',
 'n',
 'a',
 'm',
 'o',
 'e',
 'ñ',
 'i',
 'r',
 'l',
 ' ',
 'm',
 'g',
 ',',
 'o',
 'u',
 'a',
 'l',
 ' ',
 'e',
 'p',
 't',
 ' ',
 ' ',
 ' ',
 'c',
 'n',
 '.',
 'l',
 'e',
 'o',
 'e',
 'a',
 'o',
 'l',
 'a',
 's',
 'o',
 'e',
 'e',
 'r',
 ',',
 'a',
 'a',
 'd',
 'v',
 '

In [9]:
## Obtenemos la longitud de las sentencias  
n_sentences=len(rawX)
n_sentences

690386

Como el Quijote es muy largo y tiene muchas secuencias, puede encontrar problemas de memoria. Por ello, elegija un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero tenga cuidado porque, a menos datos, peor calidad del modelo.

In [10]:
#trabajamos en el modelo con la longitud total de las sentencias (el texto tiene 690386 oraciones de longuitud 40 caracteres)
MAX_SEQUENCES = n_sentences

# Permutar aleatoriamente una secuencia
perm = np.random.permutation(len(rawX))

# Definiremos nuestras variables "rawX - sentences" y "rawY - próximos caracteres" a los límites estípulados en MAX_SEQUENCES

# crearemos arrays desde las variables (rawX) y (rawY)
rawX, rawy = np.array(rawX), np.array(rawy) 
# permutaremos valores en los arrays mediante la variable "perm" que aplica al array la función .random.permutation de numpy
rawX, rawy = rawX[perm], rawy[perm]
# redimensionar las variables al número límite de oraciones establecido  
rawX, rawy = list(rawX[:MAX_SEQUENCES]), list(rawy[:MAX_SEQUENCES])

# mostrar número de sentencias en "rawX" después de la limitación 
print(len(rawX))

690386


<div style="color:white;
       display:fill;
       border-radius:5px;
       font-size:150%;
       font-family:Avenir;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:black;">
        3.3. Obtencion de input X y output y para el modelo
</div>

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a cree los arrays de datos X e y que pasará a su modelo.

Para ello, utilice *one-hot encoding* para el caracteres. Por ejemplo, si sólo tiene 4 caracteres (a, b, c, d), las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1).

De este modo, **X** tendrá shape *(num_sequences, seq_length, num_chars)* e **y** tendrá shape *(num_sequences, num_chars)*. 



In [11]:
# Crearemos los arrays en X y Y para alimentar el modelo mediante one_hot encoding 

# Definimos variables X y Y para crear función de codificación con one_hot encoding
X = np.zeros((len(rawX), SEQ_LENGTH , len(chars)))
y = np.zeros((len(rawX), len(chars)))

# crearemos función para cada oración en "rawX"
for i, sentence in enumerate(rawX):
    # para cada caracter en las oraciones
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[rawy[i]]] = 1

In [12]:
X[:1]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]])

In [13]:
y[:1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

<div style="color:white;
       display:fill;
       border-radius:5px;
       font-size:200%;
       font-family:Avenir;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:black;">
        4. Definicion del modelo y su entrenamiento
</div>

Una vez tiene todo,  defina el modelo. Defina un modelo que utilice una **LSTM** con **128 unidades internas**. Si bien el modelo puede definirse de una manera más compleja, para empezar debería bastar con una LSTM más una capa Dense con el *softmax* que predice el siguiente caracter a producir. Adam puede ser una buena elección de optimizador.

Una vez el modelo esté definido, entrénelo un poco para asegurarse de que la loss es decreciente. 

In [14]:
# definiremos nuestor modelo

# tipo de modelo = Sequential de Keras
model= Sequential()

# adherir capa LSTM con 128 unidades internas, tamaño de input 35, dimensión (73 caracteres . chars)
model.add(LSTM(64, input_shape=(SEQ_LENGTH, len(chars))))

# adherir capa de Dropout
model.add(Dropout(0.5))

# adherir capa densa de salida con activación softmax
model.add(Dense(len(chars), activation= "softmax"))

# mostrar gráfico del modelo creado
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                32256     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 61)                3965      
                                                                 
Total params: 36221 (141.49 KB)
Trainable params: 36221 (141.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# compilar modelo (optimizador, función de pérdida y métrica de evaluación)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
# entrenar modelo con tamaño de lote  = 256, número de épocas = 20 y mostrar progreso de entrenamiento (verbose = 2)
#entrenamiento realizado en imac M1 con 8 nucleos de cpu y 8 gb de ram
model.fit(X, y, batch_size=256, epochs=20, verbose=1)

Epoch 1/20
2697/2697 [==============================] - 92s 34ms/step - loss: 2.4170 - accuracy: 0.2956
Epoch 2/20
2697/2697 [==============================] - 97s 36ms/step - loss: 2.1238 - accuracy: 0.3570
Epoch 3/20
2697/2697 [==============================] - 96s 35ms/step - loss: 2.0374 - accuracy: 0.3805
Epoch 4/20
2697/2697 [==============================] - 97s 36ms/step - loss: 1.9789 - accuracy: 0.3966
Epoch 5/20
2697/2697 [==============================] - 95s 35ms/step - loss: 1.9363 - accuracy: 0.4082
Epoch 6/20
2697/2697 [==============================] - 94s 35ms/step - loss: 1.9036 - accuracy: 0.4169
Epoch 7/20
2697/2697 [==============================] - 93s 35ms/step - loss: 1.8790 - accuracy: 0.4242
Epoch 8/20
2697/2697 [==============================] - 94s 35ms/step - loss: 1.8590 - accuracy: 0.4302
Epoch 9/20
2697/2697 [==============================] - 93s 35ms/step - loss: 1.8428 - accuracy: 0.4342
Epoch 10/20
2697/2697 [==============================] - 94s 35m

<div style="color:white;
       display:fill;
       border-radius:5px;
       font-size:200%;
       font-family:Avenir;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:black;">
        5. Generacion de texto
</div>

Para ver cómo evoluciona Nuestro modelo del lenguaje,  generaremos un texto según va entrenando. Para ello, programeremos una función que, utilizando el modelo en su estado actual, genere texto, con la idea de ver cómo se va generando texto al entrenar cada epoch.

En el código de abajo puede ver una función auxiliar para obtener valores de una distribución multinomial. Esta función se usará para muestrear el siguiente carácter a utilizar según las probabilidades de la salida de softmax (en vez de tomar directamente el valor con la máxima probabilidad, obtiene un valor aleatorio según la distribución de probabilidad dada por softmax, de modo que los resultados serán más diversos, pero seguirán teniendo "sentido" ya que el modelo tenderá a seleccionar valores con más probabilidad).



In [16]:
def sample(probs, temperature=1.0):
    """Nos da el índice del elemento a elegir según la distribución
    de probabilidad dada por probs.
    
    Args:
      probs es la salida dada por una capa softmax:
        probs = model.predict(x_to_predict)[0]
      
      temperature es un parámetro que nos permite obtener mayor
        "diversidad" a la hora de obtener resultados. 
        
        temperature = 1 nos da la distribución normal de softmax
        0 < temperature < 1 hace que el sampling sea más conservador,
          de modo que sampleamos cosas de las que estamos más seguros
        temperature > 1 hace que los samplings sean más atrevidos,
          eligiendo en más ocasiones clases con baja probabilidad.
          Con esto, tenemos mayor diversidad pero se cometen más
          errores.
    """
    # Cast a float64 por motivos numéricos
    probs = np.asarray(probs).astype('float64')
    
    # logaritmo de probabilidades y aplicamos reducción
    # por temperatura.
    probs = np.log(probs) / temperature
    
    # Volvemos a aplicar exponencial y normalizamos de nuevo
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)
    
    # Hacemos el sampling dadas las nuevas probabilidades
    # de salida (ver doc. de np.random.multinomial)
    samples = np.random.multinomial(1, probs, 1)
    return np.argmax(samples)


Utilizando la función anterior y el modelo entrenado, añadir un callback a al modelo para que, según vaya entrenando, se vean los valores que resultan de generar textos con distintas temperaturas al acabar cada epoch.

Para ello, abajo tienedisponible el callback *on_epoch_end*. Esta función elige una secuencia de texto al azar en el texto disponible en la variable
text y genera textos de longitud *GENERATED_TEXT_LENGTH* según las temperaturas en *TEMPERATURES_TO_TRY*, utilizando para ello la función *generate_text*.



In [17]:
TEMPERATURES_TO_TRY = [0.2, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 300

def generate_text(seed_text, model, length=300, temperature=1, max_length=30):
    """Genera una secuencia de texto a partir de seed_text utilizando model.
    
    La secuencia tiene longitud length y el sampling se hace con la temperature
    definida.
    """
    
    # Aquí guardaremos nuestro texto generado, que incluirá el
    # texto origen
    generated = seed_text
    
    # Utilizar el modelo en un bucle de manera que generemos
    # carácter a carácter. Habrá que construir los valores de
    # X_pred de manera similar a como hemos hecho arriba, salvo que
    # aquí sólo se necesita una oración
    # Nótese que el x que utilicemos tiene que irse actualizando con
    # los caracteres que se van generando. La secuencia de entrada al
    # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
    # incluya el último caracter predicho.
 
    prediction = []    
    
    for i in range(length):
        # Crear array de numpy para almacenar semilla generadora del texto
        X = np.zeros((1, len(generated), len(chars) ))
        
        # Codificar con one_hot encoding la secuencia semilla
        for t, char in enumerate(seed_text):
            X[0, t, char_indices[char]] = 1
        
        # Generar predicción para el próximo caracter desde la secuencia semilla
        preds = model.predict(X, verbose=0)[0]
        # Elegir un caracter desde las probabilidades de predicción
        next_index = sample(preds,0.2)
        next_char = indices_char[next_index]
        
        # Adherir el cacarter predicho a las secuencias semilla 
        prediction.append(next_char)
        seed_text = seed_text[1:] + next_char

        print(next_char, end= " ");
        # Flush so we can see the prediction as it's generated
        sys.stdout.flush()

    prediction = ''.join(prediction)
    sys.stdout.flush()
    
    ### FIN DE TU CÓDIGO
    return generated

# definir función para generar texto luego de cada época
def on_epoch_end(epoch, logs):
  print("\n\n\n")
  
  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
  seed_text = text[start_pos:start_pos + SEQ_LENGTH]
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(
        epoch + 1, temperature))
    
    generated_text = generate_text(seed_text, model, 
                                   GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))

# creamos callback para adherir al entrenamiento y visualizar el texto generado tras cada época
generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)  

Entrenamos nuestro modelo añadiendo *generation_callback* a la lista de callbacks utilizados en fit(). Ya que las métricas de clasificación no son tan críticas aquí (no nos importa tanto acertar el carácter exacto, sino obtener una distribución de probabilidad adecuada), no es necesario monitorizar la accuracy ni usar validation data, si bien puedes añadirlos para asegurarte de que todo está en orden.


In [18]:
# entrenamos el modelo adicionando el callback para mostrar el texto tras cada época
model.fit(X, y, batch_size=256, epochs=20, callbacks=generation_callback)

Epoch 1/20
2697/2697 [==============================] - ETA: 0s - loss: 2.4284 - accuracy: 0.2890



------> Epoch: 1 - Generando texto con temperature 0.2
  c o n   d e   l a   p o r   e n   a s   l a   l a s   d e   d e   l o   s e   c o s   s e   c o n   e n   d e   l a   l a   d e   l a   p o n   e n   d e   l a   m o n   d e   s e   q u e   l a   c o s   l a   m o n   a l   c o n   d e   l a   l a   d e   c o n   l a   l a   d e   c o n   d e   l o   l e   c o n   e n   d e   l o   l a   l a   d e   s u   l o   d e   s u n a   a   l o   s e   l o   l a   l a   c o r   a   l a   d e s   l a s   s e   c a n   a l   l o   l o   l a   d e   l a   d e   s e   l a   l o   p o r   e n   o s   l a   s e   l a   l a   l a   l a   l a   l a   p o n   Seed: ho. pues si tú
sabes que tienes mujer re
Texto generado: ho. pues si tú
sabes que tienes mujer re
------> Epoch: 1 - Generando texto con temperature 0.5
  l o   s e   c o n   e n   a l   m o s   l a   l a   l a   a n   p o r   e n   e n  

r   s e   l a   s e   c a n t a   d e   s u   c a r a   d e   l a   m e   c o n t e   a   d e   m a n t o   d e   l a   c o n   e n   l a   a m e r a   c o n   e n   l a   c o n t e   d e   l a   c a b a l l e r o   d e   l a   c a n t a   d e   l a   c a n t e   d e   m e   l a   c a b a l l e r a   a   l a   q u e   l a   c o n t e   d e   l a   c o n   e n   l a   c o n t e   d e   l a   d e s t a   d e   l a   c a b a l l e r o   d e   l a   c o n t a s   d e   l a   s e   l e   l a   t a n t a   d e   l a   d e s t a s   d e   l a   c a n t e   d e   l a   c o n   c o n   e n   e n   l a Seed: inea tenga salud y
contento, nosotros po
Texto generado: inea tenga salud y
contento, nosotros po
------> Epoch: 3 - Generando texto con temperature 1.2
r   d e   l a   c u e n t o   d e   l a   c u a l l a   d e   l a   c o n t a   d e   c o n   e s t a   s e ñ o r   q u e   l a   s u   d e s   q u e   a   c o n   l a   d e s t a n   a   l a   s e   l a   d e   l a   c o n   l a   c o n   l

  p o r   s i   l a   m e n t e r   d e   l a   c a b a l l e r o   d e   l a   m e n t a d o   d e   l a   l a s   t i e n t a s   d e   l a   m e r d a d   d e   l a   c u a l   e n   e l   d e   l a   m e r a d o   d e   l a   d e   l a   d e   s u   c a b a l l e r o   d e   l a   t a n t a   d e   l a   c a b a l l e r o   d e   s u   d e   s e   l a   c a b a l l e r o   d e   s u   c a b a l l e r o   d e   l a   c a b a l l e r o   l a   s u s   e n t r a s   l a   d e s t a   d e   l a   c a b a l l e r o   d e   l a   c a b a l l e r o   d e   s u   c a b a l l e r o   d e   l a   c Seed: nestos principios merecen.

-¡ay señora!
Texto generado: nestos principios merecen.

-¡ay señora!
------> Epoch: 6 - Generando texto con temperature 0.5
  y   e n   l a   l a s   t e n t e s   d e   l a   c a b a l l e r o   d e   l a   c a b a l l e r o   d e   l a   m e n t e r   d e   s e   e n t a n t a   d e   l a   c u e n t o   d e   l o s   c o n t e n t e s   d e   l a   c a b a l l

l a   m e r o s   a l   c a n t o   e n   e n t r e   a   l a   m e r c e d   d e   l a   c a b a l l e r o   a   l a   m e r d e   d e   l a   d e   l a   c a b a l l e r o   a   l a   d e   l a   c a b a l l e r o   l a   m e n t e r   d e   l a   c o n t e r a   a   d e   l a   m e r c e d   d e   l a   c o n t e r a   d e   l a   d e   l a   c a n t e r a   d e   l a   m e n t e r   a   l a   c a n t e   a   l a   m e n t a r   a   l a   c a b a l l e r o   d e   m u s t o   d e   l a   c o n t e r a   d e   l a   d e   l a   d e   l a s   a l g a n t e s ,   y   e n   l a   m e r a d a   Seed: s quería; pero él no tomó ninguno, sino 
Texto generado: s quería; pero él no tomó ninguno, sino 
------> Epoch: 8 - Generando texto con temperature 1.2
d e   s u   c a n t a r   e n   l a   c a n t e   d e   l a   m e r d e   d e   l a   d e s t a   d e   l a   v e n t a   d e   l a   c a b a l l e r o   a   l a   c a b a l l e r o   d e   l a   c u a l   d e   l a   c a b a l l e r o   d e

m e r c e d   d e   l a   d e   l a   s e r   d e   l a   m e r c e d   d e   l a   m a n o   d e   l a   s e ñ o r o   d e   l a   c u e s t o   y   e l   c a b a l l e r o   a   d e   l a   c a b a l l e r o   d e   l a   m e r d e d   d e   l a   m e r d e d   d e   s u   d e c i r   e l   m u n c i a   d e   l a s   d e   l a s   a l m a n o s   d e   l a   c a n t a d o   d e   l a   c a m a   d e   l a   c a b a l l e r o   d e   l a   d e s t a   l o s   d e   l a   c a b a l l e r o   d e   l a   c a b a l l e r o   d e   l o s   a c a b a s   q u e   l e   s e ñ o r   e n   l a   m e Seed: n igualarse los tesoros que encierra la

Texto generado: n igualarse los tesoros que encierra la

------> Epoch: 11 - Generando texto con temperature 0.5
m e r c e d   d e   s u   c o n t i d a   y   d e   s u   c a b a l l e r o   e n   e l   c a b a l l e r o   e n   e l   m a n d o r   q u e   s e   h a b í a   c a b a l l e r o   d e   s u   d e s t a   d e   h a b í a   d e   l a   c a 

o r   l a   d e   l a   c o r t e n t e   d e   l a   d e   a l g u n a   d e   l a   c a r a   d e   l a   c a r t e   d e   l a   m e r c e d   d e   l a   c o n t e n t e   d e   l a   c a b a l l e r o ,   y   d e   l a   c a r t e   d e   l a   c a b a l l e r o   d e   s u   c a b a l l e r o   d e   l a   d e   l a   m e r c e d   d e   l a   c a b a l l e r o   d e   s u   d e   c o n t e n t o   d e   l a   d e   s u   s e ñ o r   d e   l a   m e r c e d   d e   l a   c o n t i e r a   d e   l a   d e   l a   d e   l a   m e n t e r   a   a l g u n a   d e   l o s   d e   t a n t o   Seed: e no era suya la culpa. replicó sancho p
Texto generado: e no era suya la culpa. replicó sancho p
------> Epoch: 13 - Generando texto con temperature 1.2
o r   d e   l a   m e r c e d   d e   l a   c a r a   d e   l a   c a r t e   d e   l a   d e   l a   c a b a l l e r o   d e   s u   m e r c e d   d e   l a   m e r c e d   s e   l a   d e   l a   m e r c e d   d e   l a   c a r t e   d e 

  d e   l a   m e n c e d   t o d o   e n   e l   m e r c e d   d e   l a   c a b a l l e r o   d e   s u   m e r c e d   d e   l a   e s t a d o   d e   l a   c a b a l l e r o   d e   l a   s e ñ o r   d e   l a   c a r t e   d e   l a   t a n t o   d e   l a   c a r t e   d e   c o n t e n t o   d e   l a   m e r d e d   d e   l a   c a r t a   d e   l a   m e n c e d   d e   l a   c a r t e   d e   l a   m e r c e d   a l   m e r c e d   d e   l a   m a n o   d e   l a   m e n c e d   l a   c a r t e   e n   e l   c a r t e   d e   l a   c a r t e   e n   e l   c a b a l l e r o   e n   l Seed: jo el
regidor que le había visto al otro
Texto generado: jo el
regidor que le había visto al otro
------> Epoch: 16 - Generando texto con temperature 0.5
  d e   l a   c a r t a   q u e   e s   a   l a   d e   l a   m a l a   d e   l a   m a n a   d e   l o s   a   l a   d e   s u   c a r t e   d e   l a   c a b a l l e r o   a   l a   c a r t e   d e   l a   s e ñ o r   q u e   s e   s e   

e r t a r   d e   s e   l e   s e r   e l   d e   l a   c o n t e n t a   d e   l a   c a r t e   d e   l a   d e   a l g u n a   d e   l a   c o n t e n t a   d e   l a   c u a l   a   l a   c o m p a r t e   e n   l a   c a l l a   d e   l a   c a r t e   q u e   e l   c u a l   l a   d e   l a   c a l a d a   d e   l a   c a b a l l e r o   d e   l a   c o n t i n a   d e   l a   d e   l a   c a r t e   q u e   s e   e s t a   d e   l a   c o n t e n d o   d e   s e   l a   a l g u n a   d e   c o n t e n t o   d e   l a   c a b a l l e r o   d e   l a   c o n t e n t a   d e   l a   c a m Seed: o y salir buscando las aventuras;
y, ens
Texto generado: o y salir buscando las aventuras;
y, ens
------> Epoch: 18 - Generando texto con temperature 1.2
e r t a d o   a   l a   c a r t e   d e   l a   c a r t a   d e   l a   c a b a l l e r o   d e   l a   m a n o   d e   l a   s e ñ o r a   e n   e l   c u a l   m e   d e c i r   d e   s u   c a m a   l a   c o n t r a   d e   l a   c a b 

<div style="color:white;
       display:fill;
       border-radius:5px;
       font-size:200%;
       font-family:Avenir;
       letter-spacing:0.5px">
    <p style="padding: 10px;
          color:black;">
        6. Entregable
</div>

El objetivo no es conseguir generar pasajes literarios con coherencia, sino obtener lenguaje que se asemeje en cierta manera a lo visto en el texto original y donde las palabras sean reconocibles como construcciones en castellano. Como ejemplo de lo que se puede conseguir, este es el resultado de generar texto después de 10 epochs y con temperature 0.2:

Un apunte interesante del entrenamiento del modelo que encontramos es que cuando entrenamos el modelo con la funcion callback, el estres al que se sometio el computador disminuyo notablemente, ya que ejecutar la funcion callback aunque demora cierto tiempo, no exige tantos recursos de la cpu, y permite al computador bajar un poco la temperatura de funcionamiento antes de iniciar la siguiente epoca de entrenamiento en nuestro caso se midio la temperatura promedio del procesador en los dos procesos (con y sin funcion callback) y hubo una diferencia de mas de 7 grados (de 83° a 90°), es decir como conclusion, en un mac, si el vector de temperaturas es grande (mas de 4 posiciones) el computador aunque demorara mas, se estresara menos y por tanto trabajara a temperaturas mas bajas.

```
------> Epoch: 20 - Generando texto con temperature 0.2
Seed: o quise decir dijo dorotea. y esto ll
Texto generado: o quise decir dijo dorotea. y esto llegado en la merced de la de la merced le desponder de la menter de la merced de la carta de la mender de la casto de la cara de la merced de la caballero de la merced de la mencedo de la menter de la de la merced de la mondar en la merced de la ventar de los de su carte de la menter de la caballero



------> Epoch: 20 - Generando texto con temperature 0.5
Seed: o quise decir dijo dorotea. y esto ll
Texto generado: o quise decir dijo dorotea. y esto llevar de la contante de la mencia de la carte de la mano de la menter de la merced y por la carte de la carte de la carte de la asimante de la merced de la mancha que se la de alguna de la caballero de la mano de la mano que de la verte de la mista de la caballero de la merced de la carte de la mano 


------> Epoch: 20 - Generando texto con temperature 1.0
Seed: o quise decir dijo dorotea. y esto ll
Texto generado: o quise decir dijo dorotea. y esto llega que se la de su desto me contenta que esta de la señora de la mender que de esto de la caballero de la merced de la carte de la mentar de la para de la mano y de la merced en la merced en la mender de la caballero de la merced de lo que a l caballero de la cuerto de la contanto de la misto de la


------> Epoch: 20 - Generando texto con temperature 1.2
Seed: o quise decir dijo dorotea. y esto ll
Texto generado: o quise decir dijo dorotea. y esto llegar de la carte que se la mano de la merced que se estaba en esta contento de la montera de la carte de la munda de la ser de la carte de la mencedo en lo que esta de la camo de la menter de la para de la merced en la caballero de la merced de la señor de la caballero de la corte de la cuerto de la

```
Con los resultados obtenidos se observa que a temperaturas bajas, el algoritmo tiende a repetir mucho las palabras, en cambio con temperaturas mas altas, se intenta formar frases con diferentes palabras y en algunas veces tiene sentido sobre todo si la frase es corta, cuanto mas larga es la frase mas sentido va perdiendo la prediccion.

Con la longitud del seed con que estamos generando el texto, el algoritmo no parece lograr desarrollar una frase extensa y con sentido, ya que se observa que al principio de la prediccion parece tener algo de sentido pero a medida que las palabras aumentas la oracion empieza a carecer de coherencia



Bibliografia

---

<li>Campesato, O. (2021). Chapter 7. Transformer . BERT, and GPT. En: Natural language processing fundamentals for developers [versión electrónica]. (pp. 247-287). Mercury Learning and information.</li>
<li>Ganegedara, T. (2018). Defining inputs in TensorFlow. En: Natural language processing with TensorFlow: Teach language to machines using python's deep learning library [version electrónica]. (pp. 37-152). Packt Publishing.</li>
<li>Srinivasa, B. (2018). Chapter 13. Deep learning for Text. En: Natural language processing and computational linguistics: A practical guide to text analysis with Python, Gensim, Spacy, and Keras. [versión electrónica]. (pp. 224-236). Packt Publishing.</li>
<li>Zhou, Y. (2022). Natural Language Processing with Improved Deep Learning Neural Networks. Scientific Programming, Scientific Programming, 2022, 1–8.</li>